- Vamos criar reglin e regpol para a base fornecida utilizando os seguintes regressores:

experience_level
employment_type

remote_ratio
company_location
company_size

- Determinar qual a melhor regressão que podemos utilizar para esses dados.

- No teste de hipótese, usar shapiro e levene.

In [250]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import statsmodels.api as sm
import scipy.stats .stats as stats
from sklearn.preprocessing import PolynomialFeatures


In [251]:
df = pd.read_csv('salaries.csv')

In [252]:
df.head(2)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2024,SE,FT,AI Engineer,90000,USD,90000,AE,0,AE,L
1,2024,SE,FT,Machine Learning Engineer,180500,USD,180500,US,0,US,M


In [253]:
df.drop(['work_year', 'job_title', 'salary_currency', 'salary_in_usd', 'employee_residence'], axis=1, inplace=True)
df.head(15)

,experience_level,employment_type,salary,remote_ratio,company_location,company_size
0,SE,FT,90000,0,AE,L
1,SE,FT,180500,0,US,M
2,SE,FT,96200,0,US,M
3,SE,FT,235000,0,AU,M
4,SE,FT,175000,0,AU,M
5,MI,FT,95413,100,US,M
6,MI,FT,70692,100,US,M
7,SE,FT,110000,0,UA,M
8,SE,FT,48000,0,UA,M
9,SE,FT,222300,0,US,M


In [254]:
# Transformar coluna com variável categórica em variável binária (dummy) df

df = pd.get_dummies(df, columns=['experience_level', 'employment_type', 'company_location', 'company_size'])
df.head()


,salary,remote_ratio,experience_level_EN,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_CT,employment_type_FL,employment_type_FT,employment_type_PT,...,company_location_SI,company_location_TH,company_location_TR,company_location_UA,company_location_US,company_location_VN,company_location_ZA,company_size_L,company_size_M,company_size_S
0,90000,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,180500,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
2,96200,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
3,235000,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,175000,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [255]:
# Dividir minha base total em teste e treino

#saber quantas linhas
#df.count()
#13972

# dividir minha base total em base de treino - usar metade
df_train = df.sample(6986)

#retirar da base principal os meus treinos
df = df.drop(df_train.index)


In [256]:
#crio uma lista com 10 amostras e 50 observações cada a apartir das 200 obs que sobraram
# após a retirada das obs de treino. HAVERÁ REPETIÇÃO
samples = []
for i in range(10):
  samples.append(df.sample(50))

  print(samples)

A saída de streaming foi truncada nas últimas 5000 linhas.
16                       0               0               1               0  
2810                     0               0               1               0  
2828                     0               0               1               0  
5492                     0               0               1               0  
6634                     0               0               1               0  

[50 rows x 90 columns],         salary  remote_ratio  experience_level_EN  experience_level_EX  \
6745    160000             0                    0                    0   
13531   167000           100                    0                    0   
10208   120000             0                    0                    0   
967      92300             0                    0                    0   
4084    295490             0                    0                    0   
5490    119200           100                    1                    0   
11103    6000

In [257]:
df.head()

,salary,remote_ratio,experience_level_EN,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_CT,employment_type_FL,employment_type_FT,employment_type_PT,...,company_location_SI,company_location_TH,company_location_TR,company_location_UA,company_location_US,company_location_VN,company_location_ZA,company_size_L,company_size_M,company_size_S
1,180500,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
2,96200,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0
3,235000,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,175000,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
5,95413,100,0,0,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,0


In [258]:
X_train = df_train[[ 'remote_ratio', 'experience_level_EN', 'experience_level_EX',
 'experience_level_MI', 'experience_level_SE', 'employment_type_CT',
 'employment_type_FL', 'employment_type_FT', 'employment_type_PT',
 'company_location_AD', 'company_location_AE', 'company_location_AM',
 'company_location_AR', 'company_location_AS', 'company_location_AT',
 'company_location_AU', 'company_location_BA', 'company_location_BE',
 'company_location_BR', 'company_location_BS', 'company_location_CA',
 'company_location_CF', 'company_location_CH', 'company_location_CL',
 'company_location_CN', 'company_location_CO', 'company_location_CZ',
 'company_location_DE', 'company_location_DK', 'company_location_DZ',
 'company_location_EC', 'company_location_EE', 'company_location_EG',
 'company_location_ES', 'company_location_FI', 'company_location_FR',
 'company_location_GB', 'company_location_GH', 'company_location_GI',
 'company_location_GR', 'company_location_HK', 'company_location_HN',
 'company_location_HR', 'company_location_HU', 'company_location_ID',
 'company_location_IE', 'company_location_IL', 'company_location_IN',
 'company_location_IQ', 'company_location_IR', 'company_location_IT',
 'company_location_JP', 'company_location_KE', 'company_location_KR',
 'company_location_LB', 'company_location_LT' , 'company_location_LU',
 'company_location_LV', 'company_location_MD', 'company_location_MT',
 'company_location_MU', 'company_location_MX', 'company_location_MY',
 'company_location_NG', 'company_location_NL', 'company_location_NO',
 'company_location_NZ', 'company_location_OM', 'company_location_PH',
 'company_location_PK', 'company_location_PL', 'company_location_PR',
 'company_location_PT', 'company_location_QA', 'company_location_RO',
 'company_location_RU', 'company_location_SA', 'company_location_SE',
 'company_location_SG', 'company_location_SI', 'company_location_TH',
 'company_location_TR', 'company_location_UA', 'company_location_US',
 'company_location_VN', 'company_location_ZA', 'company_size_L',
 'company_size_M', 'company_size_S']].to_numpy()
#variável de interesse
Y_train = df_train[['salary']].to_numpy()

In [259]:
# Criei modelo de regressão linear

model_linear = LinearRegression()
model_linear.fit(X_train, Y_train)
score_linear = model_linear.score(X_train, Y_train)
print(score_linear)

0.7840731764676396


In [260]:
poly_features = PolynomialFeatures(degree=2)
X_poly_train = poly_features.fit_transform(X_train)
model_poly = LinearRegression()
model_poly.fit(X_poly_train, Y_train)
score_polynomial = model_poly.score(X_poly_train, Y_train)
print(score_polynomial)

0.8534922157539461


In [261]:
# Começar a treinar o modelo

test_score_linear = []
test_score_polynomial = []
for sample in samples:
  #Criando variaveis X, Y para regressao linear
  X_test = sample [[ 'remote_ratio', 'experience_level_EN', 'experience_level_EX',
 'experience_level_MI', 'experience_level_SE', 'employment_type_CT',
 'employment_type_FL', 'employment_type_FT', 'employment_type_PT',
 'company_location_AD', 'company_location_AE', 'company_location_AM',
 'company_location_AR', 'company_location_AS', 'company_location_AT',
 'company_location_AU', 'company_location_BA', 'company_location_BE',
 'company_location_BR', 'company_location_BS', 'company_location_CA',
 'company_location_CF', 'company_location_CH', 'company_location_CL',
 'company_location_CN', 'company_location_CO', 'company_location_CZ',
 'company_location_DE', 'company_location_DK', 'company_location_DZ',
 'company_location_EC', 'company_location_EE', 'company_location_EG',
 'company_location_ES', 'company_location_FI', 'company_location_FR',
 'company_location_GB', 'company_location_GH', 'company_location_GI',
 'company_location_GR', 'company_location_HK', 'company_location_HN',
 'company_location_HR', 'company_location_HU', 'company_location_ID',
 'company_location_IE', 'company_location_IL', 'company_location_IN',
 'company_location_IQ', 'company_location_IR', 'company_location_IT',
 'company_location_JP', 'company_location_KE', 'company_location_KR',
 'company_location_LB', 'company_location_LT' , 'company_location_LU',
 'company_location_LV', 'company_location_MD', 'company_location_MT',
 'company_location_MU', 'company_location_MX', 'company_location_MY',
 'company_location_NG', 'company_location_NL', 'company_location_NO',
 'company_location_NZ', 'company_location_OM', 'company_location_PH',
 'company_location_PK', 'company_location_PL', 'company_location_PR',
 'company_location_PT', 'company_location_QA', 'company_location_RO',
 'company_location_RU', 'company_location_SA', 'company_location_SE',
 'company_location_SG', 'company_location_SI', 'company_location_TH',
 'company_location_TR', 'company_location_UA', 'company_location_US',
 'company_location_VN', 'company_location_ZA', 'company_size_L',
 'company_size_M', 'company_size_S']].to_numpy()
  Y_test = sample [['salary']].to_numpy()
  test_score_linear.append(model_linear.score(X_test, Y_test))

 # Criando variavel polinomial para o X
X_test_poly = poly_features.fit_transform(X_test)
test_score_polynomial.append(model_poly.score(X_test_poly, Y_test))

alpha = 0.1

In [264]:
#Modelagem do teste de hipóteses
# H0: score de teste é igual ao score de treino
# a minha regressão não perde poder de explicação quando testa novas variávies
# H1: score de teste é diferente  do score de treino
# a minha regressão não explica a variável de interesse da mesma maneira que o teste
t_stat, p_valor = stats.ttest_1samp(test_score_linear, popmean=score_linear)
print("print score linear:", score_linear)
print("print test score linear:",test_score_linear)
if p_valor < alpha:
  print('Regressão Linear: Aceite H1 e rejeito H0')
else:
  print('Regressão Linear: Aceito H0 e rejeito H1')

t_stat, p_valor = stats.ttest_1samp(test_score_polynomial, popmean=score_polynomial)
print("print score polinomial:", score_polynomial)
print("print test score polinomial:", test_score_polynomial)
if p_valor < alpha:
  print('Regressão Polinomial: Aceite H1 e rejeito H0')
else:
  print('Regressão Polinomial: Aceito H0 e rejeito H1')

print score linear: 0.7840731764676396
print test score linear: [0.3581221904451902, -1140502269103796.5, -6.099567964282555, 0.8256717987912451, 0.3876292228869451, -5.137609185688067e+16, -0.450480355039528, -8.526586428187542e+16, -4.296124225089823e+16, -567400248002426.9]
Regressão Linear: Aceite H1 e rejeito H0
print score polinomial: 0.8534922157539461
print test score polinomial: [-29569130072047.707]
Regressão Polinomial: Aceito H0 e rejeito H1


<ipython-input-264-037dc0ac0937>:6: DeprecationWarning: Please use `ttest_1samp` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  t_stat, p_valor = stats.ttest_1samp(test_score_linear, popmean=score_linear)
<ipython-input-264-037dc0ac0937>:14: DeprecationWarning: Please use `ttest_1samp` from the `scipy.stats` namespace, the `scipy.stats.stats` namespace is deprecated.
  t_stat, p_valor = stats.ttest_1samp(test_score_polynomial, popmean=score_polynomial)


O melhor modelo é o de Regressão Linear, apesar do R-quadrado menor, rejeita o H0, ou seja, o p_valor é menor que o alpha.